In [ ]:
# we want to fine-tune bart_lfqa on mixed dataset in CL approach

In [1]:
!pip install transformers datasets accelerate tqdm torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (41

In [ ]:
# loading datasets
from datasets import load_dataset
lfqa = load_dataset('fahdsoliman/lfqa_dataset')

support_doc = load_dataset('vblagoje/lfqa_support_docs')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/300 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/300 [00:00<?, ? examples/s]

Repo card metadata block was not found. Setting CardData to empty.


Generating train split:   0%|          | 0/226147 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3020 [00:00<?, ? examples/s]

In [ ]:
lfqa

DatasetDict({
    train: Dataset({
        features: ['answers', 'q_id', 'document', 'answers_urls', 'url', 'selftext', 'selftext_urls', 'title', 'subreddit', 'title_urls'],
        num_rows: 3000
    })
    validation: Dataset({
        features: ['answers', 'q_id', 'document', 'answers_urls', 'url', 'selftext', 'selftext_urls', 'title', 'subreddit', 'title_urls'],
        num_rows: 300
    })
    test: Dataset({
        features: ['answers', 'q_id', 'document', 'answers_urls', 'url', 'selftext', 'selftext_urls', 'title', 'subreddit', 'title_urls'],
        num_rows: 300
    })
})

In [2]:
import math
import re

In [ ]:
def cleanup_references(text):
    # URL reference where we need to remove both the link text and URL
    # ...and this letter is used by most biographers as the cornerstone of Lee's personal
    # views on slavery ([1](_URL_2_ & pg=PA173), [2](_URL_1_), [3](_URL_5_)).
    # ...and this letter is used by most biographers as the cornerstone of Lee's personal views on slavery.
    result = re.sub(r"[\(\s]*\[\d+\]\([^)]+\)[,)]*", "", text, 0, re.MULTILINE)

    # URL reference where we need to preserve link text but remove URL
    # At the outbreak of the Civil War, [Leyburn left his church](_URL_19_) and joined the South.
    # At the outbreak of the Civil War, Leyburn left his church and joined the South.
    result = re.sub(r"\[([^]]+)\]\([^)]+\)", "\\1", result, 0, re.MULTILINE)

    # lastly remove just dangling _URL_[0-9]_ URL references
    result = re.sub(r"_URL_\d_", "", result, 0, re.MULTILINE)
    return result

In [ ]:
def clean_answer(text):
    result = cleanup_references(text)
    result = result.replace("\n", " ")
    result = re.sub(r"\s\s+", " ", result)
    result = re.sub(r"BULLET::::-", "", result)
    return result.strip()


def clean_question(text):
    result = cleanup_references(text)
    result = result.replace("\n", " ")
    result = re.sub(r"\s\s+", " ", result)
    result = result.replace("[deleted]", "")
    return result.lower().strip()


def prepare_support_docs(example):
    provenances = example["output"][-1]["provenance"]
    context = "<P> " + " <P> ".join([p["text"] for p in provenances])
    return {"context": context}

In [ ]:
def preprocess_eli5(examples, **fn_kwargs):
    document_cache = fn_kwargs["document_cache"]
    training = fn_kwargs.get("training", True)
    extra_answer_threshold = fn_kwargs.get("extra_answer_threshold", 100)
    include_selftext = fn_kwargs.get("include_selftext", False)
    exclude_answer_patterns = fn_kwargs.get("exclude_answer_patterns", [])

    questions, contexts, answers = [], [], []
    for q_id, question, selftext, answer in zip(examples["q_id"], examples["title"], examples["selftext"],
                                                examples["answers"]):
        accepted_answer_idx = []
        if training:
            accepted_answer_idx = [idx for idx, score in enumerate(answer["score"]) if
                                   score > extra_answer_threshold]
        if not training or not accepted_answer_idx:
            accepted_answer_idx = [0]
        document = document_cache[q_id]
        for idx in accepted_answer_idx:
            skip_answer = any([p.search(answer["text"][idx]) for p in exclude_answer_patterns])
            if skip_answer:
                continue
            if include_selftext:
                questions.append(clean_question(f"{question} {selftext}"))
            else:
                questions.append(clean_question(question))
            contexts.append(document.lower().strip())
            answers.append(clean_answer(answer["text"][idx]))

    return {"question": questions, "context": contexts, "answer": answers}

In [ ]:
from datasets import load_dataset

# تحميل مجموعة البيانات
dataset = load_dataset("fahdsoliman/lfqa_with_supports_subset")

processed_datasets = {}
for split in ["train", "validation"]:
    processed_datasets[split] = dataset[split].map(
        preprocess_eli5,
        batched=True,
        desc=f"Preparing {split} dataset",
        fn_kwargs={
            "training": split == "train",
            "exclude_answer_patterns": [re.compile("not sure what you"), re.compile("\n\n >")]
        }
    )
# # تحويل البيانات إلى كائن DatasetDict
# processed_datasets_dict = DatasetDict(processed_datasets)

# # حفظ البيانات إلى ملف على القرص (اختياري)
# processed_datasets_dict.save_to_disk("./processed_datasets")


Generating train split:   0%|          | 0/371 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/56 [00:00<?, ? examples/s]

Preparing train dataset:   0%|          | 0/371 [00:00<?, ? examples/s]

KeyError: 'document_cache'

In [ ]:
from tqdm.auto import tqdm
from accelerate import Accelerator
accelerator = Accelerator()

processed_datasets = {}
support_docs_prepared = {}
with accelerator.main_process_first():
    for split in ["train", "validation"]:
        support_docs_prepared[split] = support_doc[split].map(prepare_support_docs,
                                                                batched=False,
                                                                desc="Preparing support docs",
                                                                )
    column_names = lfqa["train"].column_names
    for split in ["train", "validation"]:
        d_cache = dict([(e["id"], e["context"]) for e in tqdm(support_docs_prepared[split],
                                                              desc=f"Adding support docs to LFQA {split}")])
        processed_datasets[split] = lfqa[split].map(preprocess_eli5,
                                                    batched=True,
                                                    remove_columns=column_names,
                                                    desc="Preparing dataset for tokenization",
                                                    fn_kwargs={"document_cache": d_cache,
                                                                "training": split == "train",
                                                                "exclude_answer_patterns": [re.compile("not sure what you"),
                                                                                            re.compile("\n\n >")]}
                                                    )


Adding support docs to LFQA train:   0%|          | 0/226147 [00:00<?, ?it/s]

Preparing dataset for tokenization:   0%|          | 0/3000 [00:00<?, ? examples/s]

Adding support docs to LFQA validation:   0%|          | 0/3020 [00:00<?, ?it/s]

Preparing dataset for tokenization:   0%|          | 0/300 [00:00<?, ? examples/s]

In [ ]:
processed_datasets

{'train': Dataset({
     features: ['question', 'context', 'answer'],
     num_rows: 3004
 }),
 'validation': Dataset({
     features: ['question', 'context', 'answer'],
     num_rows: 286
 })}

In [7]:
from huggingface_hub import login

# تسجيل الدخول باستخدام رمز الوصول الشخصي
login()  # استبدل YOUR_HF_TOKEN برمز الوصول الخاص بك

In [ ]:
from datasets import DatasetDict

# تأكد من أن preprocessed_dataset هو من نوع DatasetDict
preprocessed_dataset_dict = DatasetDict(processed_datasets)

# اسم المستخدم والمستودع على Hugging Face
repo_id = "fahdsoliman/lfqa_with_supports_subset"

# رفع البيانات إلى المستودع
preprocessed_dataset_dict.push_to_hub(repo_id, private=False)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/502 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/fahdsoliman/lfqa_with_supports_subset/commit/40f11e37390f515fbf7e516fd204224cf26eac07', commit_message='Upload dataset', commit_description='', oid='40f11e37390f515fbf7e516fd204224cf26eac07', pr_url=None, pr_revision=None, pr_num=None)

In [3]:
from datasets import load_dataset
eli5 = load_dataset('fahdsoliman/NAITS_LFQA_with_supports')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/60 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/25 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/17 [00:00<?, ? examples/s]

In [4]:
def cleanup_references(text):
    # URL reference where we need to remove both the link text and URL
    # ...and this letter is used by most biographers as the cornerstone of Lee's personal
    # views on slavery ([1](_URL_2_ & pg=PA173), [2](_URL_1_), [3](_URL_5_)).
    # ...and this letter is used by most biographers as the cornerstone of Lee's personal views on slavery.
    result = re.sub(r"[\(\s]*\[\d+\]\([^)]+\)[,)]*", "", text, 0, re.MULTILINE)

    # URL reference where we need to preserve link text but remove URL
    # At the outbreak of the Civil War, [Leyburn left his church](_URL_19_) and joined the South.
    # At the outbreak of the Civil War, Leyburn left his church and joined the South.
    result = re.sub(r"\[([^]]+)\]\([^)]+\)", "\\1", result, 0, re.MULTILINE)

    # lastly remove just dangling _URL_[0-9]_ URL references
    result = re.sub(r"_URL_\d_", "", result, 0, re.MULTILINE)
    return result

def clean_answer(text):
    result = cleanup_references(text)
    result = result.replace("\n", " ")
    result = re.sub(r"\s\s+", " ", result)
    result = re.sub(r"BULLET::::-", "", result)
    return result.strip()

def clean_question(text):
    result = cleanup_references(text)
    result = result.replace("\n", " ")
    result = re.sub(r"\s\s+", " ", result)
    result = result.replace("[deleted]", "")
    return result.lower().strip()

def prepare_support_docs(example):
    related_texts = example["related_text"]
    context = "<P> " + " <P> ".join([text["text"] for text in related_texts])
    return {"context": context}

def preprocess_custom(examples, **fn_kwargs):
    questions, contexts, answers = [], [], []
    for question, related_text, answer in zip(examples["question"], examples["related_text"], examples["answer"]):
        context = "<P> " + " <P> ".join([doc["translate"] for doc in related_text])
        questions.append(clean_question(question))
        contexts.append(context.lower().strip())
        answers.append(clean_answer(answer))
    return {"question": questions, "context": contexts, "answer": answers}



In [5]:
from accelerate import Accelerator

accelerator = Accelerator()

naits_processed_datasets = {}
with accelerator.main_process_first():
    for split in ["train", "validation", "test"]:
        naits_processed_datasets[split] = eli5[split].map(preprocess_custom,
                                                    batched=True,
                                                    remove_columns=eli5[split].column_names,
                                                    desc="Preparing dataset for tokenization")

Preparing dataset for tokenization:   0%|          | 0/60 [00:00<?, ? examples/s]

Preparing dataset for tokenization:   0%|          | 0/25 [00:00<?, ? examples/s]

Preparing dataset for tokenization:   0%|          | 0/17 [00:00<?, ? examples/s]

In [6]:
naits_processed_datasets

{'train': Dataset({
     features: ['answer', 'question', 'context'],
     num_rows: 60
 }),
 'validation': Dataset({
     features: ['answer', 'question', 'context'],
     num_rows: 25
 }),
 'test': Dataset({
     features: ['answer', 'question', 'context'],
     num_rows: 17
 })}

In [8]:
from datasets import DatasetDict

# تأكد من أن preprocessed_dataset هو من نوع DatasetDict
preprocessed_dataset_dict = DatasetDict(naits_processed_datasets)

# اسم المستخدم والمستودع على Hugging Face
repo_id = "fahdsoliman/naits_lfqa_processed"

# رفع البيانات إلى المستودع
preprocessed_dataset_dict.push_to_hub(repo_id, private=False)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/493 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/fahdsoliman/naits_lfqa_processed/commit/c90eac2bed48668011911d333a39423bbbc49f9c', commit_message='Upload dataset', commit_description='', oid='c90eac2bed48668011911d333a39423bbbc49f9c', pr_url=None, pr_revision=None, pr_num=None)

here we finish prepare the datasets and we move forword to fine-tuning

In [ ]:
# الأن سنقوم بتدريب النموذج على البيانات من مجموعتي البيانات القديمة والجديدة بشكل مختلط

In [ ]:
from sklearn.utils import shuffle
from datasets import load_dataset

naits_lfqa = load_dataset('fahdsoliman/naits_lfqa_processed')
old_lfqa = load_dataset('fahdsoliman/lfqa_with_supports_subset')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/16 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/369 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/59 [00:00<?, ? examples/s]

In [ ]:
# دمج البيانات بشكل مختلط
from datasets import concatenate_datasets
combined_data = {}
for split in ['train', 'validation']:
  combined_data[split] = concatenate_datasets([old_lfqa[split], naits_lfqa[split]])
  combined_data[split] = combined_data[split].shuffle(seed=42)

In [ ]:
combined_data['validation'][12]

{'question': 'i have shared hosting that i reserved a year ago, but i received an email saying that this hosting has been suspended due to non-payment of the extension fees. i want to know whether the hosting will be canceled if the extension fees are not paid.',
 'context': '<p> article (32) hosting renewal: renewal fees shall be paid before the end of the service. the authority is not responsible for any damage or loss resulting from stopping the hosting service or not renewing it if the customer does not pay the due fees. article (33) hosting renewal: in the case of public sector entities, the authority sends a financial claim one month before the renewal due date. article (34) hosting renewal: hosting is renewed annually after receiving notice of payment of fees. article (35) hosting modification: the customer can request to modify the current type of hosting. provided that the customer pays the difference in fees if the new hosting fees are higher than the current hosting fees, an

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorWithPadding, get_scheduler
from torch.utils.data import DataLoader
from accelerate import Accelerator
import torch

accelerator = Accelerator()
model_name = "vblagoje/bart_lfqa"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

In [ ]:
import numpy as np
label_pad_token_id = -100
def tokenize_dataset(examples):
    inputs = ["question: {} context: {}".format(q, c) for q, c in zip(examples["question"], examples["context"])]
    targets = examples["answer"]
    model_inputs = tokenizer(inputs, max_length=1024, padding="max_length", truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=360, padding=True, truncation=True,
                            return_tensors="np")

    model_inputs["decoder_input_ids"] = labels["input_ids"][:, :-1].tolist()
    # replace pad_token_id with label_pad_token_id to avoid loss calculation on those tokens
    labels["input_ids"] = np.where(labels["input_ids"] == tokenizer.pad_token_id,
                                    label_pad_token_id, labels["input_ids"])

    model_inputs["labels"] = labels["input_ids"][:, 1:].tolist()
    return model_inputs

# تحويل البيانات باستخدام دالة التحويل
# tokenized_train = train_dataset.map(preprocess_function, batched=True)
# tokenized_eval = eval_dataset.map(preprocess_function, batched=True)
tokenized_datasets = {}
for split in ["train", "validation"]:
        tokenized_datasets[split] = combined_data[split].map(tokenize_dataset,
                                                    batched=True,
                                                    remove_columns=combined_data[split].column_names,
                                                    desc="Preparing dataset for tokenization")

Preparing dataset for tokenization:   0%|          | 0/385 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Preparing dataset for tokenization:   0%|          | 0/63 [00:00<?, ? examples/s]

In [ ]:
train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["validation"]
train_dataset.set_format(type='torch')
eval_dataset.set_format(type='torch')

In [ ]:
# إنشاء DataCollator لتجميع البيانات
data_collator = DataCollatorWithPadding(tokenizer)

# إعداد DataLoader للتدريب والاختبار
train_dataloader = DataLoader(tokenized_datasets["train"], shuffle=True, batch_size=2, collate_fn=data_collator)
eval_dataloader = DataLoader(tokenized_datasets["validation"], shuffle=False, batch_size=2, collate_fn=data_collator)

# إعداد النموذج والتدريب باستخدام Accelerator
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, torch.optim.AdamW(model.parameters(), lr=2e-4), train_dataloader, eval_dataloader
)

# جدولة التعلم
num_training_steps = len(train_dataloader) * 3  # عدد العصور
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)


In [ ]:
# تدريب النموذج
model.train()
for epoch in range(3):
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
    print(f"Epoch {epoch + 1} completed.")

In [ ]:
# تقييم النموذج
model.eval()
eval_metrics = []
for batch in eval_dataloader:
    with torch.no_grad():
        outputs = model(**batch)
    eval_metrics.append(accelerator.gather_for_metrics(outputs.loss))
print("Evaluation completed.")

In [ ]:
print(tokenized_datasets['validation'][0])

{'input_ids': [0, 40018, 35, 115, 5, 776, 467, 9, 5853, 102, 15167, 28, 1687, 25, 17579, 116, 5377, 35, 28696, 642, 15698, 17579, 3092, 8, 2956, 21179, 24644, 1537, 50, 285, 4902, 33, 13412, 187, 46157, 4, 5, 866, 9, 5, 155, 2586, 3220, 741, 1755, 9131, 6801, 260, 15167, 9, 9473, 493, 34, 57, 1602, 25, 22, 102, 592, 1538, 30010, 113, 8, 22, 102, 2345, 9, 194, 30886, 845, 24, 34, 57, 1695, 578, 18401, 39298, 11416, 578, 6025, 89, 58, 4785, 9, 17579, 802, 11, 5, 2302, 9, 15855, 821, 18230, 44267, 2968, 3938, 8, 38939, 23592, 4, 475, 1222, 417, 677, 5, 3240, 36, 417, 2550, 740, 4, 195, 1978, 50, 195, 2517, 8635, 238, 10, 20187, 811, 20460, 43835, 12, 19027, 661, 6, 28014, 20460, 16869, 8, 20300, 5, 285, 205, 4, 4091, 41757, 19480, 6166, 1076, 12, 4147, 1594, 18195, 338, 43724, 6, 10, 15625, 9, 35375, 14701, 44800, 6, 16, 11223, 30, 171, 25, 10, 5402, 20761, 7618, 1342, 9, 16, 5112, 636, 30886, 4, 5, 29500, 9, 1236, 293, 687, 5, 7319, 7138, 9, 5, 29224, 811, 6825, 32, 5705, 6263, 25, 17579

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=3,
    load_best_model_at_end=False,
    eval_steps = 50,
    # save_steps=50,
    logging_steps=10
)

# دالة حساب المقاييس
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # حساب المقاييس هنا (مثل Rouge، BLEU، إلخ)
    # يمكن استخدام مكتبات مثل datasets metric لهذا الغرض
    from datasets import load_metric
    metric = load_metric("rouge")
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    return {key: value.mid.fmeasure * 100 for key, value in result.items()}

# إعداد المدرب
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
# تحميل نتائج التدريب
training_logs = trainer.state.log_history

# عرض سجل المعلومات
for log in training_logs:
    print(log)


{'train_runtime': 691.8317, 'train_samples_per_second': 1.669, 'train_steps_per_second': 0.421, 'total_flos': 2503005815439360.0, 'train_loss': 2.4283285763665163, 'epoch': 3.0, 'step': 291}


In [ ]:
# اختبار النموذج المدرب والأساسي

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForSeq2SeqLM

model_name2 = "vblagoje/bart_lfqa"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer2 = AutoTokenizer.from_pretrained(model_name2)
model2 = AutoModelForSeq2SeqLM.from_pretrained(model_name2)
model2 = model.to(device)

In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# it all starts with a question/query
query = "Why does water heated to room temperature feel colder than the air around it?"

# given the question above suppose these documents below were found in some document store
documents = ["when the skin is completely wet. The body continuously loses water by...",
             "at greater pressures. There is an ambiguity, however, as to the meaning of the terms 'heating' and 'cooling'...",
             "are not in a relation of thermal equilibrium, heat will flow from the hotter to the colder, by whatever pathway...",
             "air condition and moving along a line of constant enthalpy toward a state of higher humidity. A simple example ...",
             "Thermal contact conductance In physics, thermal contact conductance is the study of heat conduction between solid ..."]

# concatenate question and support documents into BART input
conditioned_doc = "<P> " + " <P> ".join([d for d in documents])
query_and_docs = "question: {} context: {}".format(query, conditioned_doc)

model_input = tokenizer2(query_and_docs, truncation=True, padding=True, return_tensors="pt")

generated_answers_encoded = model2.generate(input_ids=model_input["input_ids"].to(device),
                                           attention_mask=model_input["attention_mask"].to(device),
                                           min_length=64,
                                           max_length=256,
                                           do_sample=False,
                                           early_stopping=True,
                                           num_beams=8,
                                           temperature=1.0,
                                           top_k=None,
                                           top_p=None,
                                           eos_token_id=tokenizer.eos_token_id,
                                           no_repeat_ngram_size=3,
                                           num_return_sequences=1)
tokenizer2.batch_decode(generated_answers_encoded, skip_special_tokens=True,clean_up_tokenization_spaces=True)



['The water in the water heater is warmer than the air around it. This is because there is more convection within the water than between the water and the air. The convection is what makes it feel colder, because the molecules in the air are moving faster than the water molecules, so they are moving away from each other faster.']

In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# it all starts with a question/query
# it all starts with a question/query
query = "Why does water heated to room temperature feel colder than the air around it?"

# given the question above suppose these documents below were found in some document store
documents = ["when the skin is completely wet. The body continuously loses water by...",
             "at greater pressures. There is an ambiguity, however, as to the meaning of the terms 'heating' and 'cooling'...",
             "are not in a relation of thermal equilibrium, heat will flow from the hotter to the colder, by whatever pathway...",
             "air condition and moving along a line of constant enthalpy toward a state of higher humidity. A simple example ...",
             "Thermal contact conductance In physics, thermal contact conductance is the study of heat conduction between solid ..."]



# concatenate question and support documents into BART input
conditioned_doc = "<P> " + " <P> ".join([d for d in documents])
query_and_docs = "question: {} context: {}".format(query, conditioned_doc)

model_input = tokenizer(query_and_docs, truncation=True, padding=True, return_tensors="pt")

generated_answers_encoded = model.generate(input_ids=model_input["input_ids"].to(device),
                                           attention_mask=model_input["attention_mask"].to(device),
                                           min_length=64,
                                           max_length=256,
                                           do_sample=False,
                                           early_stopping=True,
                                           num_beams=8,
                                           temperature=1.0,
                                           top_k=None,
                                           top_p=None,
                                           eos_token_id=tokenizer.eos_token_id,
                                           no_repeat_ngram_size=3,
                                           num_return_sequences=1)
tokenizer.batch_decode(generated_answers_encoded, skip_special_tokens=True,clean_up_tokenization_spaces=True)


['The water in the room is warmer than the air around it. The air around the water is colder because of the conduction of the heat from the water to the air. This conduction causes the temperature difference between the water and the air to be higher than the temperature of the water. This is why when you heat water to room temperature it feels colder than when you cool it to absolute zero.']